Use the De Casteljau algorithm with SLERP on 4 given rotations.

Re-parameterize by rotation angle (i.e. twice the arc-length).

In [ ]:
from IPython.display import HTML

In [ ]:
from matplotlib.animation import FuncAnimation
import matplotlib.pyplot as plt
import numpy as np

In [ ]:
from unit_quaternion import Quaternion, UnitQuaternion, slerp

In [ ]:
from splines import ConstantSpeedAdapter

In [ ]:
from helper import angles2quat, plot_rotations, prepare_axis, update_plot

In [ ]:
class BezierSegment:
    
    def __init__(self, q0, q1, q2, q3):
        self.q0 = q0
        self.q1 = q1
        self.q2 = q2
        self.q3 = q3
        self.grid = [0, 1]
        
    def evaluate(self, t, n=0):
        if not 0 <= t <= 1:
            raise ValueError(f'invalid t: {t}')
        if n == 0:
            slerp_1_2 = slerp(self.q1, self.q2, t)
            return slerp(
                slerp(slerp(self.q0, self.q1, t), slerp_1_2, t),
                slerp(slerp_1_2, slerp(self.q2, self.q3, t), t),
                t)
        elif n == 1:
            slerp_1_2 = slerp(self.q1, self.q2, t)
            one = slerp(slerp(self.q0, self.q1, t), slerp_1_2, t)
            two = slerp(slerp_1_2, slerp(self.q2, self.q3, t), t)
            x, y, z = (two * one.inverse()).log()
            # NB: twice the angle, times 3 because degree 3
            return x * 2 * 3, y * 2 * 3, z * 2 * 3
        else:
            raise NotImplementedError(f'invalid n: {n}')

In [ ]:
b1 = BezierSegment(
    angles2quat(0, 0, 0),
    angles2quat(10, 0, 0),
    angles2quat(80, 0, 0),
    angles2quat(90, 0, 0),
)

In [ ]:
b1.grid

In [ ]:
b1.evaluate(0), b1.evaluate(0.5), b1.evaluate(1)

In [ ]:
b1.evaluate(0, 1)

In [ ]:
b1.evaluate(0.5, 1)

In [ ]:
b1.evaluate(1, 1)

In [ ]:
b2 = ConstantSpeedAdapter(b1)

In [ ]:
b2.grid

In [ ]:
b2.evaluate(0)

In [ ]:
b2.evaluate(b2.grid)

In [ ]:
def generate_rotations(b1, b2):
    max_angle = b.angle(1)
    for t, angle in zip(np.linspace(0, 1, 100), np.linspace(0, max_angle, 100)):
        yield (
            b1.evaluate(t),
            b2.evaluate(angle))

In [ ]:
def generate_rotations(*args):
    for params in zip(*(np.linspace(a.grid[0], a.grid[-1], 100) for a in args)):
        yield [a.evaluate(param) for a, param in zip(args, params)]

In [ ]:
fig, ax = plt.subplots(subplot_kw=dict(projection='dumb3d'))
collections = prepare_axis(2, ax=ax)
plt.close(fig)

In [ ]:
def ani_func(rot):
    return update_plot(collections, rot)

In [ ]:
ani = FuncAnimation(fig, ani_func, init_func=lambda: None, frames=generate_rotations(b1, b2), interval=30)
display(HTML(ani.to_jshtml(default_mode='reflect')))

In [ ]:
b3 = BezierSegment(
    angles2quat(0, 0, 0),
    angles2quat(0, 90, 0),
    angles2quat(90, -45, 0),
    angles2quat(90, 0, 0),
)

In [ ]:
b4 = ConstantSpeedAdapter(b3)

In [ ]:
fig, ax = plt.subplots(subplot_kw=dict(projection='dumb3d'))
collections = prepare_axis(2, ax=ax)
plt.close(fig)

In [ ]:
def ani_func(rot):
    return update_plot(collections, rot)

In [ ]:
ani = FuncAnimation(fig, ani_func, init_func=lambda: None, frames=generate_rotations(b3, b4), interval=30)
display(HTML(ani.to_jshtml(default_mode='reflect')))

In [ ]:
b5 = BezierSegment(
    angles2quat(0, 0, 0),
    angles2quat(10, 0, -179),
    angles2quat(20, 90, 179),
    angles2quat(30, 90, 0),
)

In [ ]:
b6 = ConstantSpeedAdapter(b5)

In [ ]:
fig, ax = plt.subplots(subplot_kw=dict(projection='dumb3d'))
collections = prepare_axis(2, ax=ax)
plt.close(fig)

In [ ]:
def ani_func(rot):
    return update_plot(collections, rot)

In [ ]:
ani = FuncAnimation(fig, ani_func, init_func=lambda: None, frames=generate_rotations(b5, b6), interval=30)
display(HTML(ani.to_jshtml(default_mode='reflect')))